In [1]:
import cv2 
import numpy as np 
import mtcnn 
from model_architecture import *
from sklearn.preprocessing import Normalizer
from scipy.spatial.distance import cosine
from tensorflow.keras.models import load_model
import pickle

In [2]:
confidence_t=0.99
recognition_t=0.5
required_size = (160,160)

In [3]:
def normalize(img):
    mean, std = img.mean(), img.std()
    return (img - mean) / std

def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode

def load_pickle(path):
    with open(path, 'rb') as f:
        encoding_dict = pickle.load(f)
    return encoding_dict

In [9]:
def detect(img ,detector,encoder,encoding_dict):
    """Detect And Return Result
    Parameters
    ----------
    img : str
        Image path you want to predict.
    detector : MTCNN object
        MTCNN object constructor
        ex : detector = mtcnn.MTCNN()
    encoder : model_object
        Its our pretrained weight
        ex : encoder = InceptionResNetV2().load_weights(weight_path)
    encoding_dict : pickle_load_object
        ex : encoding_dict = load_pickle(encodings_path)
    """
    l2_normalizer = Normalizer('l2')
    img = cv2.imread(img)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Detect Face In Image
    results = detector.detect_faces(img_rgb)
    
    encode = get_encode(encoder, img_rgb, required_size)
    encode = l2_normalizer.transform(encode.reshape(1, -1))[0]

    name = "unknown"

    distance = float("inf")
    for db_name, db_encode in encoding_dict.items():
        dist = cosine(db_encode, encode)
        # Jika semakin dekat or "dist is low" the accuracy is high
        if dist < recognition_t and dist < distance:
                name = db_name
                distance = dist

    if name == "unknown":
        print("Doesnt recognize")
    else:
        print("This is {} with Distance {}".format(name, distance))
    
    return img


In [7]:
required_shape = (160,160)
face_detector = mtcnn.MTCNN()
face_encoder = InceptionResNetV2() 
weight_path = "model/facenet_keras_weights.h5"
face_encoder.load_weights(weight_path)
encoding_path = "encodings.pkl"
encoding_dict = load_pickle(encoding_path)

In [10]:
image_path = "Extracted/face.jpg"
img = detect(image_path, face_detector, face_encoder, encoding_dict)

This is Zein with Distance 0.4640985131263733
